In [1]:
from qiskit_nature.units import DistanceUnit                                  #Angstrom unit
from qiskit_nature.second_q.drivers import PySCFDriver                        #PySCF Driver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer        #ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper                 #Mappers


In [2]:
%%time
ethylene="C 0.00000000 0.00000000 0.76532900; H -0.51007400 0.88446900 1.16448900; H 1.02100900   -0.00049700    1.16448900; H -0.51093500   -0.88397100    1.16448900;  C 0.00000000 0.00000000 -0.76532900; H 0.51093500 -0.88397100 -1.164489; H -1.02100900 -0.00049700 -1.16448900; H 0.51007400 0.88446900 -1.16448900"
driver = PySCFDriver(
    
    
    atom=ethylene,
       
    
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

es_problem = driver.run()

CPU times: user 614 ms, sys: 75.9 ms, total: 690 ms
Wall time: 2.31 s


In [3]:
%%time
problem=driver.run()
print(problem)

CPU times: user 435 ms, sys: 2.6 ms, total: 438 ms
Wall time: 439 ms


In [4]:
print(problem.num_particles)
print(problem.num_spatial_orbitals)

(9, 9)
16


In [27]:
as_transformer = ActiveSpaceTransformer(2, 2, active_orbitals=[8, 9]) 


In [28]:
%%time
problem2 = as_transformer.transform(problem)

CPU times: user 43.5 ms, sys: 8 ms, total: 51.5 ms
Wall time: 1.78 s


In [7]:
print(problem2.num_particles)
print(problem2.num_spatial_orbitals)

(1, 1)
2


In [8]:
fermionic_op = problem2.hamiltonian.second_q_op()

In [9]:
%%time
qubit_jw_op = JordanWignerMapper().map(fermionic_op)

qubit_jw_op.num_qubits

CPU times: user 55.6 ms, sys: 398 µs, total: 56 ms
Wall time: 160 ms


4

In [10]:
fermionic_op.to_matrix().toarray().shape

(16, 16)

In [11]:
from qiskit_nature.second_q.mappers import QubitConverter


converter = QubitConverter(JordanWignerMapper(), z2symmetry_reduction="auto")

qubit_jw_op = converter.convert(fermionic_op, sector_locator=problem2.symmetry_sector_locator)
print(qubit_jw_op.num_qubits)

2


In [12]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD

vqe_solver = VQEUCCFactory(Estimator(), UCCSD(qubit_converter=converter), SLSQP())

In [13]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

In [14]:
%%time
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(converter, vqe_solver)

CPU times: user 16 µs, sys: 3 µs, total: 19 µs
Wall time: 22.9 µs


In [34]:
%%time
res = calc.solve(problem2)
print(res)

AlgorithmError: 'The number of qubits of the ansatz does not match the operator, and the ansatz does not allow setting the number of qubits using `num_qubits`.'

In [35]:
qubit_jw_op

TaperedPauliSumOp(SparsePauliOp(['II', 'IZ', 'IX', 'ZI', 'XI', 'ZZ', 'ZX', 'XZ', 'XX'],
              coeffs=[-4.97932605e-01+0.j, -4.12410016e-01+0.j,  2.36530495e-06+0.j,
 -4.12410016e-01+0.j,  2.36530495e-06+0.j,  2.43340865e-02+0.j,
 -2.36506888e-06+0.j, -2.36506888e-06+0.j,  2.09160604e-02+0.j]), coeff=1.0)